# Audience
Пример отчета по аудитории Web-Index.

## Описание задания

Задача посчитать аудиторию проектов:
- Avito.ru – аудитория одного проекта
- Avito.ru, Ivi.ru – аудиторию 2-х проектов, т.е. количество людей, которые посетили хотя бы один проект (Avito.ru)OR(Ivi.ru)
- Total – Total Internet

за период: Май 2020

список статистик, который будем считать:
- Unweighted Reach
- Reach
- Reach%
- Average Daily Reach
- Average Daily Reach%
- Average Weekly Reach
- Average Weekly Reach%


Ограничений по типам устройст нет, считаем по всем (Web Desktop,Web Mobile,App Online,App Offline).


Для расчета данного отчета потребуется сформировать несколько заданий для Responsum, т.к. он умеет считать только native статистики (Unweighted Reach, Reach, Average Daily Reach,...), а процентные статистики можно будет посчитать на основе полученных данных по native.
Кроме этого в отчете присутствуют кастомыне переменные (Avito.ru, Ivi.ru), а так же Total Internet - это тоже отдельные задания


## Инициализация

Первым шагом любого ноутбука для построения отчетов необходимо загрузить библиотеки, которые нам помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# получаем список демографических переменных
#demo_ds = rc.get_demo()
#demo_dict = rc.get_demo_dict(demo_ds)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

Loading BokehJS ...

In [4]:
## Формируем задания

Нам понадобиться сформировать три задания для Responsum:
- расчет аудитории для проекта Avito.ru
- расчет аудитории по проектам Avito.ru, Ivi.ru
- расчет аудитории Total Internet

SyntaxError: invalid syntax (<ipython-input-4-b55ec94b6161>, line 3)

### Общие параметры для заданий

Для начала зададим общие параметры

In [2]:
# выбираем тип установки mobile, т.к. в расчетах учавствуют все типу устройств (Web Desktop,Web Mobile,App Online,App Offline)
facility = 'mobile'
# задаем период расчета
date_from = '2020-01-01'
date_to = '2020-01-31'

rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)


../.cache/b64ddd38624a01074f8c070115df740e.cache
../.cache/0231f39eb4750320e57a2a8641218d9f.cache
../.cache/b64ddd38624a01074f8c070115df740e.cache


In [5]:
msnet.__file__

'/Users/psv/Projects/responsum-note/mediascope_api/core/net.py'

In [40]:
rnet = msnet.MediascopeApiNetwork()
data = msnet.send_raw_request('get', '/media/holdings?facility_id={}'.format(facility))    
# https://api.mediascope.net/responsum/api/v1/media/holdings?facility_id=mobile

init RepsonsumNetwork: <resp.net.RepsonsumNetwork object at 0x7f8a03cce550>


In [17]:
data[:100]

'[{"id":16351,"title":"Mindshareworld","sites":[{"id":24213,"title":"279954_apr_11_LG_SmartTV (25.04.'

### Получим ID проектов
Для продолжения расчетов, нам необходимо получить идентификаторы проектов Avito.ru и Ivi.ru из медиа-дерева

In [10]:
# Найдем все холдинги в названия которых входит название avito.ru (пойск выполняется без учета регистра)


d = rcats.get_demo()
d

../.cache/b64ddd38624a01074f8c070115df740e.cache


,varId,varName,varTitle,categories,from,to
0,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",None,None
1,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",None,None
2,148,AGE,Возраст,"[{'varId': 148, 'title': '', 'usetypes': [{'us...",None,None
3,157,Mat_pol,Материальное положение семьи,"[{'varId': 157, 'catNum': 1, 'title': 'не хват...",None,None
4,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",None,None
5,161,PERSNUM,Количество человек в семье,"[{'varId': 161, 'catNum': 1, 'title': '1', 'or...",None,None
6,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",None,None
7,171,INCOME_GROUPS,Уровень дохода семьи,"[{'varId': 171, 'catNum': 1, 'title': 'Ниже ср...",None,None
8,172,EDUC2,Образование,"[{'varId': 172, 'catNum': 1, 'title': 'н.средн...",None,None
9,173,WORK2,Занятость,"[{'varId': 173, 'catNum': 1, 'title': 'работае...",None,None


In [11]:
# Найдем все холдинги в названия которых входит название avito.ru (пойск выполняется без учета регистра)
df_holdings = rc.get_holdings(facility_id=facility, find_text='avito.ru')
df_holdings

AttributeError: module 'mediascope_api.responsum.catalogs' has no attribute 'get_holdings'

In [12]:
df_holdings

NameError: name 'df_holdings' is not defined

Из полученного результата видим, что нас интересует холдинг с __ID = 15827__

Получим подробную информацию по холдингу:

In [10]:
df_holding = rc.get_holding(facility_id=facility,id=15827)
df_holding

,profileName,messages
0,responsum_mediascope,"400 : [{""stackTrace"":[{""methodName"":""lambda$ge..."


Нас интересует Проект - он же site, из полученных данных видим, что ID проекта 

__Avito.ru = 15828__

#### Найдем ID для Ivi.ru

In [129]:
# Найдем все холдинги в названия которых входит название avito.ru (пойск выполняется без учета регистра)
df_holdings = rc.get_holdings(facility_id=facility,find_text='ivi.ru')
df_holdings

,id,title,sites,adAgency,network
82,211127,Рекламные кампании Ivi.ru,"[{'id': 363149, 'title': 'Сентябрь 2017', 'sec...",True,False
412,34496,Рекламные кампании Ivi.ru,"[{'id': 206392, 'title': 'Июль 2014', 'section...",True,False
1469,15868,Ivi.ru,"[{'id': 15869, 'title': 'Ivi.ru', 'sections': ...",False,False


Из полученного результата видим, что нас интересует холдинг с __ID = 15868__

Получим подробную информацию по холдингу:

In [135]:
df_holding = rc.get_holding(facility_id=facility,id=34496, find_text='Ноябрь')
df_holding

,id,title,site_id,site_title,section_id,section_title,subsection_id,subsection_title
211,34496,Рекламные кампании Ivi.ru,170536,Самореклама Ivi ноябрь 2013,171479,ivi_samoognipritona,83741,ivi_samoognipritona
212,34496,Рекламные кампании Ivi.ru,170536,Самореклама Ivi ноябрь 2013,170541,ivi_samoparanoidpark,82553,ivi_samoparanoidpark
213,34496,Рекламные кампании Ivi.ru,170536,Самореклама Ivi ноябрь 2013,171475,ivi_samo5nevest,83766,ivi_samo5nevest
214,34496,Рекламные кампании Ivi.ru,170536,Самореклама Ivi ноябрь 2013,170538,ivi_samooceanseleven,82530,ivi_samooceanseleven
215,34496,Рекламные кампании Ivi.ru,170536,Самореклама Ivi ноябрь 2013,170537,ivi_samomissunshine,82537,ivi_samomissunshine
...,...,...,...,...,...,...,...,...
1559,34496,Рекламные кампании Ivi.ru,56029,Ноябрь 2011,46505,Always/Always Super Premium_ivi.ru_пре-ролл_21...,17485,Always/Always Super Premium_ivi.ru_пре-ролл_21...
1560,34496,Рекламные кампании Ivi.ru,56029,Ноябрь 2011,44772,УКО-сбербанк онлайн_ivi.ru _постролл_01.11-06.11,16465,УКО-сбербанк онлайн_ivi.ru _постролл_01.11-06.11
1561,34496,Рекламные кампании Ivi.ru,56029,Ноябрь 2011,46508,Fairy/Regenerist15_ivi.ru_пре-ролл_03.11-13.11,17477,Fairy/Regenerist15_ivi.ru_пре-ролл_03.11-13.11
1562,34496,Рекламные кампании Ivi.ru,56029,Ноябрь 2011,46502,часы Ника_ivi.ru_пре-ролл_21.11-27.11,17468,часы Ника_ivi.ru_пре-ролл_21.11-27.11


Нас интересует Проект - он же site, из полученных данных видим, что ID проекта 

__Ivi.ru = 15869__

Перейдем к формированию заданий


### Задание №1 Расчет аудитории по проекту Avito.ru

In [136]:
# Задаем название задания (может пригодиться для понимания, где какое задание среди посчитанных)
task_name = 'Avito.ru may 2020'
# Задаем медиа фильтр, в нашем случае это ID проекта Avito.ru
media_filter = "site = 15828"
# Фильтр по демографии отсутствует
demo_filter = None
# Задаем список статистик для расчета (указываем только нативные)
statistics=["UnwReach", "Reach", "ADR", "AMR"]
# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rt.build_audience_task(facility=facility, date_from=date_from, date_to=date_to, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет
task_audience = rt.send_audience_task(task_json)
task_audience

{'profileName': 'responsum_mediascope',
 'taskId': '35ed4d6e-3ecc-4fdd-904e-38e4f3af7499',
 'operation': 'Расчет задачи.',
 'messages': ['Задача <35ed4d6e-3ecc-4fdd-904e-38e4f3af7499> поступила в обработку']}

#### Получаем результат

In [138]:
task_result = rt.get_result(task_audience['taskId'])
task_result

{'taskId': '35ed4d6e-3ecc-4fdd-904e-38e4f3af7499',
 'profileName': 'responsum_mediascope',
 'deliveryTypes': [],
 'messageType': 'INFO',
 'messages': [],
 'cells': [{'coord': {'dtPoint': None,
    'demoPoint': None,
    'mediaPoint': None,
    'usetypePoint': None},
   'values': {'unwreach': 9007.0,
    'reach': 39969.333508417185,
    'adr': 8942.51682317758,
    'amr': 39969.333508417214}}],
 'resultAsJson': None,
 'actual': True}

#### Преобразуем результат в DataFrame

На данном шаге, важно дать уникальное название нашем DataFrame'у, т.к. он нам понадобиться для дальнейших расчетов. Пусть будет __df_avito__


In [139]:
df_avito = rt.result2table(task_result)
df_avito

,unwreach,reach,adr,amr
0,9007.0,39969.333508,8942.516823,39969.333508


### Задание №2 Расчет аудитории по проектам Avito.ru и Ivi.ru

In [156]:
# Задаем название задания (может пригодиться для понимания, где какое задание среди посчитанных)
task_name = 'Avito.ru + Ivi.ru may 2020'
# Задаем медиа фильтр, в нашем случае это ID проектов Avito.ru и Ivi.ru
media_filter = "site = 15828 OR site = 15869"
# Фильтр по демографии отсутствует
demo_filter = None
# Задаем список статистик для расчета (указываем только нативные)
statistics=["UnwReach", "Reach", "ADR", "AMR"]
# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False,
    "media": ['site']
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rt.build_audience_task(facility=facility, date_from=date_from, date_to=date_to, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет
task_audience = rt.send_audience_task(task_json)
task_audience

{'profileName': 'responsum_mediascope',
 'taskId': '6f5b5c2c-2490-4b19-a76a-9490ecf3f092',
 'operation': 'Расчет задачи.',
 'messages': ['Задача <6f5b5c2c-2490-4b19-a76a-9490ecf3f092> поступила в обработку']}

#### Получаем результат

In [169]:
task_result = rt.get_result(task_audience['taskId'])
task_result

{'taskId': '6f5b5c2c-2490-4b19-a76a-9490ecf3f092',
 'profileName': 'responsum_mediascope',
 'deliveryTypes': [],
 'messageType': 'INFO',
 'messages': [],
 'cells': [{'coord': {'dtPoint': None,
    'demoPoint': None,
    'mediaPoint': {'type': 'site', 'val': 15869},
    'usetypePoint': None},
   'values': {'unwreach': 3277.0,
    'reach': 14271.305170426656,
    'adr': 1197.6400332212481,
    'amr': 14271.305170426669}},
  {'coord': {'dtPoint': None,
    'demoPoint': None,
    'mediaPoint': {'type': 'site', 'val': 15828},
    'usetypePoint': None},
   'values': {'unwreach': 9007.0,
    'reach': 39969.333508417185,
    'adr': 8942.51682317758,
    'amr': 39969.333508417105}}],
 'resultAsJson': None,
 'actual': True}

#### Преобразуем результат в DataFrame

На данном шаге, важно дать уникальное название нашем DataFrame'у, т.к. он нам понадобиться для дальнейших расчетов. Пусть будет __df_avito_ivi__

In [170]:
df_avito_ivi = rt.result2table(task_result)
df_avito_ivi

,site,unwreach,reach,adr,amr
0,15869,3277.0,14271.305170,1197.640033,14271.305170
1,15828,9007.0,39969.333508,8942.516823,39969.333508


### Задание №3 Расчет аудитории TotalInternet

In [148]:
# Задаем название задания (может пригодиться для понимания, где какое задание среди посчитанных)
task_name = 'Total Internet may 2020'
# Фильтр по медиа отсутствует
media_filter = None
# Фильтр по демографии отсутствует
demo_filter = None
# Задаем список статистик для расчета (указываем только нативные)
statistics=["UnwReach", "Reach", "ADR", "AMR"]
# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rt.build_audience_task(facility=facility, date_from=date_from, date_to=date_to, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет
task_audience = rt.send_audience_task(task_json)
task_audience

{'profileName': 'responsum_mediascope',
 'taskId': 'd2fa59c3-1032-4c99-b65a-74e2f6abc7f4',
 'operation': 'Расчет задачи.',
 'messages': ['Задача <d2fa59c3-1032-4c99-b65a-74e2f6abc7f4> поступила в обработку']}

#### Получаем результат

In [152]:
task_result = rt.get_result(task_audience['taskId'])
task_result

{'taskId': 'd2fa59c3-1032-4c99-b65a-74e2f6abc7f4',
 'profileName': 'responsum_mediascope',
 'deliveryTypes': [],
 'messageType': 'INFO',
 'messages': [],
 'cells': [{'coord': {'dtPoint': None,
    'demoPoint': None,
    'mediaPoint': None,
    'usetypePoint': None},
   'values': {'unwreach': 19812.0,
    'reach': 95336.20946921819,
    'adr': 85375.43601236114,
    'amr': 95336.2094692181}}],
 'resultAsJson': None,
 'actual': True}

#### Преобразуем результат в DataFrame

На данном шаге, важно дать уникальное название нашем DataFrame'у, т.к. он нам понадобиться для дальнейших расчетов. Пусть будет __df_total__

In [153]:
df_total = rt.result2table(task_result)
df_total

,unwreach,reach,adr,amr
0,19812.0,95336.209469,85375.436012,95336.209469


### Формирование итоговой таблицы

Следующим шагом соберем полученные результаты в один DataFrame, и расчитаем процентные статистики

In [154]:
# Для начала добавим название проектов в отдельный столбик
df_avito['project'] = 'avito.ru'
df_avito_ivi['project'] = 'avito.ru+ivi.ru'
df_total['project'] = 'total'

# Объединим наши DataFrame'ы
df_result = pd.concat([df_avito, df_avito_ivi, df_total])
df_result


,unwreach,reach,adr,amr,project
0,9007.0,39969.333508,8942.516823,39969.333508,avito.ru
0,10385.0,46592.158751,9982.918132,46592.158751,avito.ru+ivi.ru
0,19812.0,95336.209469,85375.436012,95336.209469,total


In [1]:
32000 * 2000

64000000

In [2]:
30000 * 1000

30000000